In [3]:
import streamlit as st
import pandas as pd
import plotly.express as px

# --- CONFIGURATION ---
# The Live Google Sheet URL you provided
SHEET_URL = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTR8Pa4QQVSNwepSe9dYnro3ZaVEpYQmBdZUzumuLL-U2IR3nKVh-_GbZeJHT2x9aCqnp7P-0hPm5Zd/pub?output=csv"

st.set_page_config(page_title="Virgo Cup CM5 Dashboard", page_icon="🏆", layout="wide")

# --- REUSABLE DATA CLEANING ---
def find_column(df, keywords):
    """Finds a column even if the name changes slightly."""
    clean_cols = df.columns.str.lower().str.replace(' ', '').str.replace('_', '')
    for i, col in enumerate(clean_cols):
        for key in keywords:
            if key in col:
                return df.columns[i]
    return None

def clean_currency(series):
    """Turns '$1,000' or '1 000' into 1000.0"""
    return (series.astype(str)
            .str.replace('$', '', regex=False)
            .str.replace(',', '', regex=False)
            .str.replace(' ', '', regex=False)
            .apply(pd.to_numeric, errors='coerce')
            .fillna(0))

def clean_percentage(series):
    """Turns '50%' into 50.0"""
    s = series.astype(str).str.replace('%', '', regex=False)
    return pd.to_numeric(s, errors='coerce').fillna(0)

def parse_uma_details(series):
    """Extracts 'Oguri Cap' from 'Oguri Cap (Christmas)'"""
    return series.astype(str).apply(lambda x: x.split('-')[0].split('(')[0].strip())

# --- DATA LOADING ---
@st.cache_data(ttl=60) # Auto-refresh every 60 seconds
def load_data():
    try:
        df = pd.read_csv(SHEET_URL)
        
        # 1. Map Columns (Robust Search)
        col_map = {
            'ign': find_column(df, ['ign', 'player', 'name', 'trainer']),
            'group': find_column(df, ['group', 'bracket', 'league']),
            'money': find_column(df, ['money', 'spent', 'cost']),
            'runs': find_column(df, ['runs', 'attempts', 'count']),
            'ace_uma': find_column(df, ['r1d1', 'uma1', 'ace']),
            'win_rate': find_column(df, ['winrate', 'win%', 'wr'])
        }

        # 2. Clean Data based on mapping
        if col_map['win_rate']: df['Clean_WinRate'] = clean_percentage(df[col_map['win_rate']])
        else: df['Clean_WinRate'] = 0.0

        if col_map['money']: df['Clean_Money'] = clean_currency(df[col_map['money']])
        else: df['Clean_Money'] = 0.0

        if col_map['runs']: df['Clean_Runs'] = pd.to_numeric(df[col_map['runs']], errors='coerce').fillna(0)
        else: df['Clean_Runs'] = 0

        if col_map['ace_uma']: df['Clean_AceName'] = parse_uma_details(df[col_map['ace_uma']])
        else: df['Clean_AceName'] = "Unknown"
        
        # Handle missing Group or IGN
        if col_map['group']: df['Clean_Group'] = df[col_map['group']].fillna("Unknown")
        else: df['Clean_Group'] = "Unknown"
        
        if col_map['ign']: df['Clean_IGN'] = df[col_map['ign']].fillna("Anonymous")
        else: df['Clean_IGN'] = "Anonymous"

        return df
    except Exception as e:
        st.error(f"Data Error: {e}")
        return pd.DataFrame()

# --- APP LOGIC ---
df = load_data()

st.title("🏆 Virgo Cup CM5 Interactive Analytics")
st.caption("Live data from Google Sheets • Auto-refreshes every 60s")

if df.empty:
    st.warning("Waiting for data... please check the CSV URL.")
else:
    # 1. Sidebar Filters
    st.sidebar.header("Filters")
    groups = list(df['Clean_Group'].unique())
    selected_groups = st.sidebar.multiselect("Select Group", groups, default=groups)
    
    if selected_groups:
        filtered_df = df[df['Clean_Group'].isin(selected_groups)]
    else:
        filtered_df = df
    
    if st.sidebar.button("🔄 Refresh Now"):
        st.cache_data.clear()
        st.rerun()

    # 2. Top Metrics
    m1, m2, m3, m4 = st.columns(4)
    m1.metric("Total Entries", len(filtered_df))
    m2.metric("Total Runs", int(filtered_df['Clean_Runs'].sum()))
    m3.metric("Avg Win Rate", f"{filtered_df['Clean_WinRate'].mean():.1f}%")
    m4.metric("Avg Spent", f"${filtered_df['Clean_Money'].mean():,.0f}")

    st.divider()

    # 3. Interactive Charts Tabs
    tab1, tab2, tab3 = st.tabs(["💸 Money vs Wins", "🐎 Meta Report", "📊 Group Stats"])

    with tab1:
        st.subheader("Does Money Buy Wins?")
        st.markdown("Hover over points to see Player Names.")
        
        # INTERACTIVE SCATTER PLOT
        fig_scatter = px.scatter(
            filtered_df,
            x="Clean_Money",
            y="Clean_WinRate",
            color="Clean_Group",
            size="Clean_Runs",
            hover_name="Clean_IGN",  # Shows IGN on hover
            hover_data=["Clean_AceName"],
            title="Money Spent vs. Win Rate",
            labels={"Clean_Money": "Money Spent ($)", "Clean_WinRate": "Win Rate (%)"},
            template="plotly_dark"
        )
        # Add a line for average win rate
        fig_scatter.add_hline(y=filtered_df['Clean_WinRate'].mean(), line_dash="dot", annotation_text="Avg Win Rate")
        st.plotly_chart(fig_scatter, use_container_width=True)

    with tab2:
        st.subheader("Top Performing Ace Umas")
        
        # Aggregate Data
        uma_stats = filtered_df.groupby('Clean_AceName').agg({
            'Clean_WinRate': 'mean',
            'Clean_Runs': 'count',
            'Clean_Money': 'mean'
        }).reset_index()
        
        # Filter: Only Umas with at least 1 run, sort by Win Rate, Top 10
        uma_stats = uma_stats[uma_stats['Clean_Runs'] > 0].sort_values('Clean_WinRate', ascending=False).head(10)

        # INTERACTIVE BAR CHART
        fig_bar = px.bar(
            uma_stats,
            x="Clean_WinRate",
            y="Clean_AceName",
            orientation='h',
            color="Clean_WinRate",
            color_continuous_scale="Viridis",
            hover_data=["Clean_Runs", "Clean_Money"],
            title="Top 10 Umas by Win Rate",
            labels={"Clean_WinRate": "Avg Win Rate (%)", "Clean_AceName": ""}
        )
        fig_bar.update_layout(yaxis={'categoryorder':'total ascending'}) # Put #1 at top
        st.plotly_chart(fig_bar, use_container_width=True)

    with tab3:
        st.subheader("Win Rate Distribution by Group")
        
        # INTERACTIVE BOX PLOT
        fig_box = px.box(
            filtered_df,
            x="Clean_Group",
            y="Clean_WinRate",
            color="Clean_Group",
            points="all", # Show individual dots too
            hover_name="Clean_IGN",
            title="Win Rate Ranges (Box Plot)",
            labels={"Clean_Group": "Group", "Clean_WinRate": "Win Rate (%)"}
        )
        st.plotly_chart(fig_box, use_container_width=True)

    # 4. Raw Data Expander
    with st.expander("📂 View Raw Data"):
        st.dataframe(filtered_df)

2025-11-22 00:24:03.432 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-22 00:24:03.433 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-11-22 00:24:03.434 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-11-22 00:24:03.435 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-22 00:24:03.825 
  command:

    streamlit run c:\Users\white\miniconda3\envs\moo\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-11-22 00:24:03.826 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-22 00:24:03.826 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored whe

In [4]:
# Run this cell to sync your changes to dashboard.py!
import os

# This command converts this notebook (dashboard.ipynb) into a python script (dashboard.py)
os.system('jupytext --to py:percent --output dashboard.py dashboard.ipynb')

print("✅ dashboard.py has been updated!")

✅ dashboard.py has been updated!
